In [10]:
import statsmodels.api as sm
import pandas as pd #librería para manejo de datos
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
import numpy as np


datos_f1=pd.read_csv('datos/modelo.csv') 
X=datos_f1.values[:,1:13]
X = X.astype('float64')
y=datos_f1.values[:,0]
y = y.astype('float64')
#print(X)
mco1 = sm.OLS(y, sm.add_constant(X)).fit()

mco1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     28.01
Date:                Tue, 23 Nov 2021   Prob (F-statistic):           4.69e-18
Time:                        10:27:57   Log-Likelihood:                -184.83
No. Observations:                  63   AIC:                             395.7
Df Residuals:                      50   BIC:                             423.5
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.5946     14.814      1.323      0.192     -10.160      49.349
x1            11.4177      1.916      5.959      0.000       7.569      15.266
x2             2.9419      0.516      5.699      0.000       1.905       3.979
x3            -2.1852      0.793     -2.755      0.008      -3.778      -0.592
x4             0.7530      0.501      1.502      0.139      -0.254       1.760
x5            -0.0915      0.061     -1.509      0.138      -0.213       0.030
x6            -0.1015      0.333     -0.305      0.762      -0.770       0.567
x7             0.0149      0.009      1.604      0.115      -0.004       0.034
x8            -0.3179      1.363     -0.233      0.817      -3.056       2.421
x9            -1.4723      1.206     -1.221      0.228      -3.894       0.949
x10            1.3061      1.320      0.989      0.327      -1.346       3.958
x11            4.1089      1.564      2.627      0.011       0.967       7.251
x12           -3.0421      0.952     -3.194      0.002      -4.955      -1.129
==============================================================================
Omnibus:                       36.889   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.817
Skew:                           1.843   Prob(JB):                     7.00e-21
Kurtosis:                       7.667   Cond. No.                     8.32e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
beta=mco1.params
e=mco1.resid
beta=np.array(mco1.params)
#print(e)
sum(e**2)/(mco1.nobs-1)
print()
# sigmagorro=(np.dot(y, y)-np.dot(beta.T, np.dot(X.T, y))) / (mco1.nobs-1)
# print(sigmagorro)

## HETEROSCEDASTICIDAD

In [12]:
GQ=sms.het_goldfeldquandt(mco1.model.endog, mco1.model.exog, split=3) #Split es para que se quede con una tercera parte o lo que queramos
print('Het-Goldfelquandt', GQ)#El SPLIT 3 no se puede poner porque debe dar un valor entero. Por defecto lo hace entre 2
#El vector te da en el 2 valor el p valor, si tu alfa es menor que el p valor significa HOMOSCEDASTICIDAD

BP=sms.het_breuschpagan(mco1.resid, mco1.model.exog)
print('Het-Breuschpagan', BP)
#Este nos decía cuanto dependía nuestros residuos al cuadrado respecto de nuestras vbles. Sólo nos interesan los
#dos ultimos valores que son el estadístico experimental y el p valor. Como es mayor que nuestro alfa (0.05) hay HOMOSCEDASTICIDAD
WH=sms.het_white(mco1.resid, mco1.model.exog)
print('Het-Whithe', WH) #Igual, nos interesan sólo los dos últimos valores y significan lo mismo que en el anterior. 



Het-Goldfelquandt (107.76684315728731, 9.446249988747399e-14, 'increasing')
Het-Breuschpagan (12.282140420325984, 0.4232949362018341, 1.0090249373457596, 0.4550016997755348)
Het-Whithe (26.17690631055308, 0.7945305319807839, 0.6247152442944426, 0.9040299420162615)


## Glejser

In [13]:
#Glejser se hace "a mano" a cada bleble
headers = datos_f1.columns
for indice in range(1, 13):
    print(headers[indice])
    z=datos_f1.values[:,indice]
    z = z.astype('float64')
    for h in [-2,-1,-0.5,0.5,1,2]:
        mcoaux=sm.OLS(abs(mco1.resid),sm.add_constant((z**h))).fit()
        pval=mcoaux.pvalues[1]
        print("h: ", h, "-> pval:", pval, " R2: ", mcoaux.rsquared)
        #Nos debemos quedar con todas porque el pValor es menor que alfa, y de ellas escogemos la de R2 más grande
        #En este caso nos da el problema el Z^2, tenemos que multiplicar mis datos (la matriz) por 1/sqrt(Z^2)
    
z = datos_f1.values[:,13].astype('float64')
mcp = sm.WLS(y, sm.add_constant(X), weights=(1/np.sqrt(z**2))).fit() #Regresión por cuadrados ponderados
mcp.summary()

Length (m)
h:  -2 -> pval: 0.6472720112044562  R2:  0.003454336455031126
h:  -1 -> pval: 0.5012497561265785  R2:  0.00744769154672098
h:  -0.5 -> pval: 0.432201283373963  R2:  0.010144482720637171
h:  0.5 -> pval: 0.30894740204941507  R2:  0.01696393114093142
h:  1 -> pval: 0.25649305594628463  R2:  0.02105654523736289
h:  2 -> pval: 0.17149822844039392  R2:  0.03043138913741228
Curva lenta
h:  -2 -> pval: 0.8708189142451439  R2:  0.0004369925358532756
h:  -1 -> pval: 0.9743900679735642  R2:  1.7033818050848915e-05
h:  -0.5 -> pval: 0.8449016815582742  R2:  0.0006323424950215317
h:  0.5 -> pval: 0.612356553793141  R2:  0.004234777315744509
h:  1 -> pval: 0.5692349710432449  R2:  0.005340267879518801
h:  2 -> pval: 0.6162996444712464  R2:  0.004141704980662464
Curva media
h:  -2 -> pval: 0.3170981195205708  R2:  0.016406608010961055
h:  -1 -> pval: 0.20116937992091732  R2:  0.026643234843875208
h:  -0.5 -> pval: 0.1668271491993237  R2:  0.03109503257292312
h:  0.5 -> pval: 0.13291663497

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.870
Model:                            WLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     28.01
Date:                Tue, 23 Nov 2021   Prob (F-statistic):           4.70e-18
Time:                        10:27:57   Log-Likelihood:                -184.84
No. Observations:                  63   AIC:                             395.7
Df Residuals:                      50   BIC:                             423.5
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.5943     14.815      1.323      0.192     -10.162      49.350
x1            11.4177      1.916      5.958      0.000       7.569      15.267
x2             2.9418      0.516      5.699      0.000       1.905       3.979
x3            -2.1851      0.793     -2.755      0.008      -3.778      -0.592
x4             0.7530      0.501      1.502      0.139      -0.254       1.760
x5            -0.0915      0.061     -1.509      0.138      -0.213       0.030
x6            -0.1015      0.333     -0.305      0.762      -0.770       0.567
x7             0.0149      0.009      1.604      0.115      -0.004       0.034
x8            -0.3177      1.363     -0.233      0.817      -3.056       2.421
x9            -1.4719      1.206     -1.221      0.228      -3.894       0.950
x10            1.3058      1.320      0.989      0.327      -1.346       3.958
x11            4.1086      1.564      2.626      0.011       0.966       7.251
x12           -3.0419      0.952     -3.194      0.002      -4.955      -1.129
==============================================================================
Omnibus:                       36.890   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.819
Skew:                           1.843   Prob(JB):                     6.99e-21
Kurtosis:                       7.667   Cond. No.                     8.32e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.32e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
GQ=sms.het_goldfeldquandt(mcp.model.endog, mcp.model.exog, split=3) #Split es para que se quede con una tercera parte o lo que queramos
print('Het-Goldfelquandt', GQ)#El SPLIT 3 no se puede poner porque debe dar un valor entero. Por defecto lo hace entre 2
#El vector te da en el 2 valor el p valor, si tu alfa es menor que el p valor significa HOMOSCEDASTICIDAD

BP=sms.het_breuschpagan(mcp.resid, mcp.model.exog)
print('Het-Breuschpagan', BP)
#Este nos decía cuanto dependía nuestros residuos al cuadrado respecto de nuestras vbles. Sólo nos interesan los
#dos ultimos valores que son el estadístico experimental y el p valor. Como es mayor que nuestro alfa (0.05) hay HOMOSCEDASTICIDAD
WH=sms.het_white(mcp.resid, mcp.model.exog)
print('Het-Whithe', WH) #Igual, nos interesan sólo los dos últimos valores y significan lo mismo que en el anterior. 


Het-Goldfelquandt (107.76684315728731, 9.446249988747399e-14, 'increasing')
Het-Breuschpagan (12.282072382664403, 0.42330026619825684, 1.0090179942002022, 0.45500765008192634)
Het-Whithe (26.175755736950734, 0.7945775452212563, 0.6246682673356732, 0.9040651000452788)
